"¿Cómo influyen los indicadores económicos y demográficos en la tasa de crecimiento de la población a nivel global?"

In [3]:
# Importación de librerías necesarias
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination, BeliefPropagation 
from pgmpy.independencies.Independencies import IndependenceAssertion
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator

from pandas import read_csv, DataFrame
import numpy as np

from IPython.core.display import display, HTML

C:\Users\lmher\AppData\Local\Temp\ipykernel_25820\2099003912.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
# Definir constantes para los nodos de la red bayesiana
LV_URB = "Nivel urbanización"                               # SP.URB.TOTL.IN.ZS
LV_EDU = "Nivel educativo"                                  # 
RT_PARO = "Tasa de paro"                                    # SL.UEM.TOTL.ZS
PNB = "Ingreso nacional bruto"                              # NY.GNP.PCAP.CD
PIB = "Producto Interior Bruto"            #GDP             # NY.GDP.PCAP.KD.ZG
GASTO_EDUCATIVO = "Gasto educativo"                         # SE.XPD.TOTL.GD.ZS
ACCESO_SALUD = "Acceso a salud"                             # SH.XPD.CHEX.PC.CD
RT_FERTILIDAD = "Tasa de fertilidad"                        # SP.DYN.TFRT.IN
RT_CRECIMIENTO = "Tasa de crecimiento de la población"      # SP.POP.GROW
RT_MORTALIDAD = "Tasa de mortalidad"                        # 

# Se crea el objeto para el model 
model = BayesianNetwork()

# Añadimos todos los nodos
nodes = [RT_CRECIMIENTO, PIB, RT_PARO, GASTO_EDUCATIVO, ACCESO_SALUD, PNB, LV_URB, RT_FERTILIDAD, RT_MORTALIDAD]
# nodes = [LV_URB, LV_EDU, RT_PARO, PNB, PIB, GASTO_EDUCATIVO, 
#          ACCESO_SALUD, RT_FERTILIDAD, RT_CRECIMIENTO, RT_MORTALIDAD]
model.add_nodes_from(nodes)

#Creamos y añadimos los caminos (origen, destino) del grafo dirigido
edges_RT_PARO = [(LV_EDU, RT_PARO), (LV_URB, RT_PARO)]
edges_PIB = [(RT_PARO, PIB), (PNB, PIB)]
edges_RT_FERTILIDAD = [(GASTO_EDUCATIVO, RT_FERTILIDAD), (ACCESO_SALUD, RT_FERTILIDAD)]
edges_RT_CRECIMIENTO = [(RT_FERTILIDAD, RT_CRECIMIENTO), (RT_MORTALIDAD, RT_CRECIMIENTO)]

model.add_edges_from(edges_RT_PARO)
model.add_edges_from(edges_PIB)
model.add_edge(PIB, GASTO_EDUCATIVO)
model.add_edges_from(edges_RT_FERTILIDAD)
model.add_edges_from(edges_RT_CRECIMIENTO)


In [15]:
years = {"min" : 2014, "max" : 2022}
df_raw = read_csv("./csv/data.csv", delimiter=',')
        
df_raw_growth = DataFrame(df_raw[:])
print("There are " + str(df_raw_growth.shape[0]) + " indicators in the dataframe.")
df_raw_growth.head()

There are 9 indicators in the dataframe.


,Indicator Name,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Population growth,-0.3,-0.1,0.1,0.2,0.4,0.7,0.5,0.1,0.8
1,GDP per capita growth,1.7,3.9,3.0,2.7,1.8,1.3,-11.6,6.3,5.0
2,Unemployment,24.4,22.1.19.6,17.2,15.3,14.1,15.5,14.8,12.9,NaN
3,Public spending on education,4.3,4.3,4.2,4.2,4.2,4.2,4.9,4.6,4.5
4,Current health spending per capita,2679.5,2349.1,2376.6,2524.7,2741.4,2716.8,2899.0,3234.3,3143.4


In [16]:
df_growth = df_raw_growth.transpose().iloc[1:]
df_growth.columns = nodes[0:9]
df_growth

,Tasa de crecimiento de la población,Producto Interior Bruto,Tasa de paro,Gasto educativo,Acceso a salud,Ingreso nacional bruto,Nivel urbanización,Tasa de fertilidad,Tasa de mortalidad
2014,-0.3,1.7,24.4,4.3,2679.5,29160.0,79.4,1.3,0.85
2015,-0.1,3.9,22.1.19.6,4.3,2349.1,28460,79.6,1.3,0.91
2016,0.1,3.0,17.2,4.2,2376.6,27570.0,79.8,1.3,0.88
2017,0.2,2.7,15.3,4.2,2524.7,27120.0,80.1,1.3,0.91
2018,0.4,1.8,14.1,4.2,2741.4,29330.0,80.3,1.3,0.91
2019,0.7,1.3,15.5,4.2,2716.8,30360.0,80.6,1.2,0.89
2020,0.5,-11.6,14.8,4.9,2899.0,27180.0,80.8,1.2,1.04
2021,0.1,6.3,12.9,4.6,3234.3,30090.0,81.1,1.2,0.95
2022,0.8,5.0,NaN,4.5,3143.4,32090.0,81.3,1.2,0.98


In [23]:
TIERS_NUM = 3

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'

def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan

def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index != 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries

new_columns = {}

for i, content in enumerate(df_growth.items()):
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    new_columns[label] = [relabel(value, boundaries) for value in series.tolist()]

df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)
df.head(10)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# disable text wrapping in output cell
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

model.cpds = []
model.fit(data=df,
          estimator=BayesianEstimator,
          prior_type="BDeu",
          equivalent_sample_size=10,
          complete_samples_only=False)

print(f'Check model: {model.check_model()}\n')
for cpd in model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd, '\n')

In [ ]:
print(f'There can be made {len(model.get_independencies().get_assertions())}',
      'valid independence assertions with respect to the all possible given evidence.')
print('For instance, any node in the network is independent of its non-descendents given its parents (local semantics):\n',
      f'\n{model.local_independencies(nodes)}\n')

def active_trails_of(query, evidence):
    active = model.active_trail_nodes(query, observed=evidence).get(query)
    active.remove(query)
    if active:
        if evidence:
            print(f'Active trails between \'{query}\' and {active} given the evidence {set(evidence)}.')
        else:
            print(f'Active trails between \'{query}\' and {active} given no evidence.')
    else:
        print(f'No active trails for \'{query}\' given the evidence {set(evidence)}.')

def markov_blanket_of(node):
    print(f'Markov blanket of \'{node}\' is {set(model.get_markov_blanket(node))}')

active_trails_of(query='FFEC', evidence=[])
active_trails_of(query='FFEC', evidence=['EC'])
active_trails_of(query='FFEC', evidence=['EC', 'CO2'])
active_trails_of(query='EI', evidence=['EC'])
active_trails_of(query='CH4', evidence=['EC', 'FFEC'])
print()
markov_blanket_of(node='Pop')
markov_blanket_of(node='EC')
markov_blanket_of(node='REC')
markov_blanket_of(node='CH4') # note: a bug in pgmpy 0.1.10 raise a KeyError here.
                              # I opened an issue and I got accepted my pull request with the bug fix:
                              # https://github.com/pgmpy/pgmpy/issues/1293
                              # https://github.com/pgmpy/pgmpy/pull/1294

In [ ]:
def independent_assertions_score_function(node):
    return len([a for a in model.get_independencies().get_assertions() if node in a.event1])

def evidence_assertions_score_function(node):
    return len([a for a in model.get_independencies().get_assertions() if node in a.event3])

def update(assertion_dict, node, score_function):
    tmp_score = score_function(node)
    if tmp_score == assertion_dict["max"]["score"]:
        assertion_dict["max"]["nodes"].append(node)
    elif tmp_score > assertion_dict["max"]["score"]:
        assertion_dict["max"]["nodes"] = [node]
        assertion_dict["max"]["score"] = tmp_score
    if tmp_score == assertion_dict["min"]["score"]:
        assertion_dict["min"]["nodes"].append(node)
    elif tmp_score < assertion_dict["min"]["score"]:
        assertion_dict["min"]["nodes"] = [node]
        assertion_dict["min"]["score"] = tmp_score

if len(nodes) > 1:
    independent_init = independent_assertions_score_function(nodes[0])
    independent_dict = {"max": {"nodes": [nodes[0]], "score": independent_init},
                       "min": {"nodes": [nodes[0]], "score": independent_init}}
    evidence_init = evidence_assertions_score_function(nodes[0])
    evidence_dict = {"max": {"nodes": [nodes[0]], "score": evidence_init},
                    "min": {"nodes": [nodes[0]], "score": evidence_init}}
    for node in nodes[1:]:
        update(independent_dict, node, independent_assertions_score_function)
        update(evidence_dict, node, evidence_assertions_score_function)

print(f'Nodes which appear most ({independent_dict["max"]["score"]} times) in independence assertions',
      f'as independent variable are:\n{set(independent_dict["max"]["nodes"])}')
print(f'Nodes which appear least ({independent_dict["min"]["score"]} times) in independence assertions',
      f'as independent variable are:\n{set(independent_dict["min"]["nodes"])}')
print(f'Nodes which appear most ({evidence_dict["max"]["score"]} times) in independence assertions',
      f'as evidence are:\n{set(evidence_dict["max"]["nodes"])}')
print(f'Nodes which appear least ({evidence_dict["min"]["score"]} times) in independence assertions',
      f'as evidence are:\n{set(evidence_dict["min"]["nodes"])}')

In [ ]:
def check_assertion(model, independent, from_variables, evidence):
    assertion = IndependenceAssertion(independent, from_variables, evidence)
    result = False
    for a in model.get_independencies().get_assertions():
        if frozenset(assertion.event1) == a.event1 and assertion.event2 <= a.event2 and frozenset(assertion.event3) == a.event3:
            result = True
            break
    print(f'{assertion}: {result}')

check_assertion(model, independent=['EI'], from_variables=['N2O'], evidence=['EC'])
check_assertion(model, independent=['EI'], from_variables=["FFEC", "REC", "GDP", "Pop", "Urb"], evidence=['EC'])
check_assertion(model, independent=['EC'], from_variables=["CH4"], evidence=['FFEC'])
check_assertion(model, independent=['EC'], from_variables=["CH4"], evidence=['FFEC', 'REC'])
check_assertion(model, independent=['FFEC'], from_variables=["REC"], evidence=['EC'])
check_assertion(model, independent=['FFEC'], from_variables=["REC"], evidence=['EC', 'CO2'])
check_assertion(model, independent=['CH4'], from_variables=["CO2"], evidence=['FFEC'])
check_assertion(model, independent=['CH4'], from_variables=["CO2"], evidence=['FFEC', 'REC'])

In [ ]:
from pgmpy.inference import VariableElimination
import time

def query_report(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    if desc:
        print(desc)
    start_time = time.time()
    print(infer.query(variables=variables,
                      evidence=evidence,
                      elimination_order=elimination_order,
                      show_progress=show_progress))
    print(f'--- Query executed in {time.time() - start_time:0,.4f} seconds ---\n')

def get_ordering(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    start_time = time.time()
    ordering = infer._get_elimination_order(variables=variables,
                                        evidence=evidence,
                                        elimination_order=elimination_order,
                                        show_progress=show_progress)
    if desc:
        print(desc, ordering, sep='\n')
        print(f'--- Ordering found in {time.time() - start_time:0,.4f} seconds ---\n')
    return ordering

def padding(heuristic):
    return (heuristic + ":").ljust(16)

def compare_all_ordering(infer, variables, evidence=None, show_progress=False):
    ord_dict = {
        "MinFill": get_ordering(infer, variables, evidence, "MinFill", show_progress),
        "MinNeighbors": get_ordering(infer, variables, evidence, "MinNeighbors", show_progress),
        "MinWeight": get_ordering(infer, variables, evidence, "MinWeight", show_progress),
        "WeightedMinFill": get_ordering(infer, variables, evidence, "WeightedMinFill", show_progress)
    }
    if not evidence:
        pre = f'elimination order found for probability query of {variables} with no evidence:'
    else:
        pre = f'elimination order found for probability query of {variables} with evidence {evidence}:'
    if ord_dict["MinFill"] == ord_dict["MinNeighbors"] and ord_dict["MinFill"] == ord_dict["MinWeight"] and ord_dict["MinFill"] == ord_dict["WeightedMinFill"]:
        print(f'All heuristics find the same {pre}.\n{ord_dict["MinFill"]}\n')
    else:
        print(f'Different {pre}')
        for heuristic, order in ord_dict.items():
            print(f'{padding(heuristic)} {order}')
        print()

infer = VariableElimination(model)

var = ['GDP']
heuristic = "MinNeighbors"
ordering = get_ordering(infer, variables=var, elimination_order=heuristic,
                        desc=f'Elimination order for {var} with no evidence computed through {heuristic} heuristic:')
query_report(infer, variables=var, elimination_order=ordering,
             desc=f'Probability query of {var} with no evidence through precomputed elimination order:')
query_report(infer, variables=var, elimination_order=list(reversed(ordering)),
             desc=f'Probability query of {var} with no evidence through dummy elimination order:')
compare_all_ordering(infer, variables=var)

var = ['CO2']
ev = {'EC': 'A: -7.05 to -0.12'}
heuristic = "MinFill"
query_report(infer, variables=var, evidence=ev, elimination_order=heuristic,
             desc=f'Probability query of {var} with evidence {ev} computed through {heuristic} heuristic:')
compare_all_ordering(infer, variables=var, evidence=ev)
heuristic = "MinNeighbors"
query_report(infer, variables=var, evidence=ev, elimination_order=heuristic,
             desc=f'Probability query of {var} with evidence {ev} computed through {heuristic} heuristic:')

In [ ]:
query_report(infer, variables=['EC'], evidence={'Pop': 'C: +0.50 to +1.99'},
             desc='Probability of energy consumption increase with population growth over +0.50%:')
query_report(infer, variables=['CO2'], evidence={'Pop': 'C: +0.50 to +1.99'},
             desc='Probability of CO2 increase with population growth over +0.50%:')
query_report(infer, variables=['CO2'], evidence={'Urb': 'C: +0.82 to +2.84'},
             desc='Probability of CO2 increase with urbanization growth over +0.82%:')
query_report(infer, variables=['CO2'], evidence={'GDP': 'C: +2.89 to +7.49'},
             desc='Probability of CO2 increase with GDP per capita growth over +2.89%:')
query_report(infer, variables=['CO2'],
             evidence={'Pop': 'C: +0.50 to +1.99', 'Urb': 'C: +0.82 to +2.84', 'GDP': 'C: +2.89 to +7.49'},
             desc='Probability of CO2 increase with:\n - population growth over +0.50%\n - urbanization growth over +0.82%\n - GDP per capita growth over +2.89%:')
query_report(infer, variables=['CO2'], evidence={'EC': 'A: -7.05 to -0.12'},
             desc='Probability of CO2 increase with energy consumption growth under -0.12%:')
query_report(infer, variables=['CO2'], evidence={'EC': 'C: +2.79 to +13.06'},
             desc='Probability of CO2 increase with energy consumption growth over +2.79%:')
query_report(infer, variables=['CH4'], evidence={'CO2': 'C: +3.81 to +17.59'},
             desc='Probability of CH4 increase with CO2 growth over +3.81%:')
query_report(infer, variables=['CH4'], evidence={'CO2': 'A: -10.20 to -0.79'},
             desc='Probability of CH4 increase with CO2 growth under -0.79%:')
query_report(infer, variables=['CH4'],
             desc='Probability of CH4 increase with no given evidence:')
query_report(infer, variables=['N2O'], evidence={'CO2': 'C: +3.81 to +17.59'},
             desc='Probability of N2O increase with CO2 growth over +3.81%:')
query_report(infer, variables=['N2O'], evidence={'CO2': 'A: -10.20 to -0.79'},
             desc='Probability of N2O increase with CO2 growth under -0.79%:')
query_report(infer, variables=['N2O'],
             desc='Probability of N2O increase with no given evidence:')
query_report(infer, variables=['FFEC', 'REC'],
             desc='Joint probability of fossil fuel energy consumption and renewable energy consumption:')
query_report(infer, variables=['CO2'], evidence={'FFEC': 'A: -2.72 to -0.40', 'REC': 'C: +11.51 to +23.95'},
             desc='Probability of CO2 increase with:\n - fossil fuel energy consumption growth under -0.40%\n - renewable energy consumption growth over +11.51%')
query_report(infer, variables=['CH4'], evidence={'FFEC': 'A: -2.72 to -0.40', 'REC': 'C: +11.51 to +23.95'},
             desc='Probability of CH4 increase with:\n - fossil fuel energy consumption growth under -0.40%\n - renewable energy consumption growth over +11.51%')
query_report(infer, variables=['N2O'], evidence={'FFEC': 'A: -2.72 to -0.40', 'REC': 'C: +11.51 to +23.95'},
             desc='Probability of N2O increase with:\n - fossil fuel energy consumption growth under -0.40%\n - renewable energy consumption growth over +11.51%')